In [1]:

import sys
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

sys.path.append("/Users/nwong/Workspace/Projects/tdt4173_project/src")

from feature_engineering.sklearn_transformers import *
from feature_engineering.store_features import *
from feature_engineering.utils import *

In [2]:
stores_train_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_train.csv"))
stores_test_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_test.csv"))
stores_extra_df = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_extra.csv"))

income_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_income_households.csv"))
age_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_age_distribution.csv"))
household_dist = set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_households_num_persons.csv"))
grunnkrets_df = combine_keys(set_year_2016(pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_norway_stripped.csv")))


bus_stops_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/busstops_norway.csv")


In [3]:
def num_households(household_dist):
    household_dist = household_dist.copy()
    population = household_dist.drop(["grunnkrets_id"], axis=1).sum(axis=1)
    household_dist["household_count"] = population
    return household_dist[["grunnkrets_id", "household_count"]]

In [4]:
def num_households_geo(geo_group, household_dist, grunnkrets_df):
    _num_households = num_households(household_dist)
    merged_df = grunnkrets_df.merge(_num_households, on="grunnkrets_id", how="inner")
    return merged_df.groupby([geo_group], as_index=False)['household_count'].sum()

In [5]:
def total_grunnkrets_income(income_dist, household_dist):
    _num_households = num_households(household_dist)
    merged_df = income_dist.merge(_num_households, on="grunnkrets_id", how="inner")[['grunnkrets_id', 'household_count', 'all_households']]
    merged_df['total_income'] = merged_df['household_count'] * merged_df['all_households']
    return merged_df[['grunnkrets_id', 'total_income']]    

In [6]:
def total_income_geo(geo_group, income_dist, household_dist, grunnkrets_df):
    grunnkrets_income = total_grunnkrets_income(income_dist, household_dist)
    merged_df = grunnkrets_df.merge(grunnkrets_income, on="grunnkrets_id", how="inner")
    return merged_df.groupby([geo_group], as_index=False)['total_income'].sum()

In [7]:
def average_household_income_geo(geo_group, income_dist, household_dist, grunnkrets_df):
    income = total_income_geo(geo_group, income_dist, household_dist, grunnkrets_df)
    households = num_households_geo(geo_group, household_dist, grunnkrets_df)
    
    merged_df = income.merge(households, on=geo_group, how="inner")
    merged_df[f'avg_household_income_{geo_group}'] = merged_df['total_income'] / merged_df['household_count']
    return merged_df[[geo_group, f'avg_household_income_{geo_group}']]
    

In [11]:
average_household_income_geo('t_district', income_dist, household_dist, grunnkrets_df)

,t_district,avg_household_income_t_district
0,AgdenesAgdenes,472900.0
1,AlfasetOslo,438900.0
2,AlgarheimUllensaker,579000.0
3,AlstadBodø,611600.0
4,Alstahaug/TjøttaAlstahaug,476900.0
...,...,...
1535,ØymarkMarker,438900.0
1536,ØyrekkenFrøya,394900.0
1537,ØysteseKvam,552500.0
1538,Øystre Slidre nordØystre Slidre,438300.0


In [12]:
average_household_income_geo('municipality_name', income_dist, household_dist, grunnkrets_df)

,municipality_name,avg_household_income_municipality_name
0,Agdenes,472900.000000
1,Alstahaug,489857.165153
2,Alta,551235.513825
3,Alvdal,480690.137107
4,Andebu,508944.273474
...,...,...
417,Østre Toten,481616.997639
418,Øvre Eiker,512771.751348
419,Øyer,477190.111248
420,Øygarden,556220.561344
